<a href="https://colab.research.google.com/github/AfifaMasood/AfifaMasood/blob/main/testing2_frame_level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)


In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import os

# Constants
IMAGE_HEIGHT, IMAGE_WIDTH = 256, 256
CLASSES_LIST = ["Normal", "Shoplifting"]

def preprocess_frame(frame):
    """
    Preprocesses a single frame for prediction.
    Resizes the frame to (IMAGE_HEIGHT, IMAGE_WIDTH) and normalizes pixel values.
    """
    resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
    return resized_frame / 255.0

def frame_predict(video_path, model, output_video_path):
    """
    Processes a video frame by frame and annotates each frame with the predicted class.
    Saves the annotated frames to a new video file.
    """
    video_capture = cv2.VideoCapture(video_path)

    # Check if video was opened successfully
    if not video_capture.isOpened():
        print(f"Error: Unable to open video file at {video_path}")
        return

    frame_count = 0

    # Get video properties
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Output codec for mp4
    frame_rate = video_capture.get(cv2.CAP_PROP_FPS)  # Frame rate of the input video
    width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Set up the VideoWriter to save the output video
    out = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (width, height))

    if not out.isOpened():
        print(f"Error: Unable to open video writer for output video at {output_video_path}")
        return

    while True:
        ret, frame = video_capture.read()
        if not ret:
            print("End of video or no frames to read.")
            break

        print(f"Processing frame {frame_count}")

        # Preprocess the current frame
        processed_frame = preprocess_frame(frame)

        # Model expects a single frame, so reshape it to (1, 256, 256, 3)
        input_data = np.expand_dims(processed_frame, axis=0)

        # Predict for the current frame
        predictions = model.predict(input_data, verbose=0)[0]
        normal_probability = predictions[0]
        shoplifting_probability = predictions[1]

        predicted_class = np.argmax(predictions)
        predicted_label = CLASSES_LIST[predicted_class]
        predicted_prob = max(normal_probability, shoplifting_probability)

        print(f"Predicted: {predicted_label} (Normal: {normal_probability:.2f}, Shoplifting: {shoplifting_probability:.2f})")

        # Annotate frame with prediction text
        text = f"Prediction: {predicted_label} ({predicted_prob:.2f})"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1.5
        font_thickness = 2
        text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
        text_width, text_height = text_size
        text_x, text_y = 10, 40

        # Draw a black rectangle as the background for the text
        cv2.rectangle(frame, (text_x - 5, text_y - text_height - 5),
                      (text_x + text_width + 5, text_y + 5), (0, 0, 0), -1)
        # Put the white text on top of the rectangle
        cv2.putText(frame, text, (text_x, text_y), font, font_scale, (255, 255, 255), font_thickness)

        # Write the annotated frame to the output video
        out.write(frame)

        frame_count += 1

        # Exit on 'q' (optional for manual control, not needed in automated processing)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    out.release()
    cv2.destroyAllWindows()


def main():
    """
    Main function to process a video and save the output.
    """
    # Path to the input video in Google Drive
    video_path = '/content/drive/MyDrive/FYP Dataset/Shoplifting/Shoplifting-24.mp4'

    # Create an output folder for the video
    output_folder = '/content/drive/MyDrive/output_folder2'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Created output folder: {output_folder}")

    # Define the output video path
    output_video_path = os.path.join(output_folder, 'Shoplifting-24_output.mp4')

    # Load the trained model
    model = tf.keras.models.load_model('/content/drive/MyDrive/60_model_bs_256_frame2.h5')  # Update with your model path

    # Process the video and save results
    frame_predict(video_path, model, output_video_path)
    print(f"Output video saved at: {output_video_path}")


if __name__ == "__main__":
    main()


Processing frame 0
Predicted: Normal (Normal: 0.85, Shoplifting: 0.15)
Processing frame 1
Predicted: Normal (Normal: 0.85, Shoplifting: 0.15)
Processing frame 2
Predicted: Normal (Normal: 0.86, Shoplifting: 0.14)
Processing frame 3
Predicted: Normal (Normal: 0.86, Shoplifting: 0.14)
Processing frame 4
Predicted: Normal (Normal: 0.86, Shoplifting: 0.14)
Processing frame 5
Predicted: Normal (Normal: 0.86, Shoplifting: 0.14)
Processing frame 6
Predicted: Normal (Normal: 0.88, Shoplifting: 0.12)
Processing frame 7
Predicted: Normal (Normal: 0.87, Shoplifting: 0.13)
Processing frame 8
Predicted: Normal (Normal: 0.88, Shoplifting: 0.12)
Processing frame 9
Predicted: Normal (Normal: 0.89, Shoplifting: 0.11)
Processing frame 10
Predicted: Normal (Normal: 0.89, Shoplifting: 0.11)
Processing frame 11
Predicted: Normal (Normal: 0.89, Shoplifting: 0.11)
Processing frame 12
Predicted: Normal (Normal: 0.91, Shoplifting: 0.09)
Processing frame 13
Predicted: Normal (Normal: 1.00, Shoplifting: 0.00)
Pr

In [ ]:
# Step 2: Define the file path
file_path = '/content/drive/My Drive/60_model_bs_256_frame2.h5'

# Step 3: Check if the file exists
if os.path.exists(file_path):
    print(f"The file '{file_path}' exists.")
else:
    print(f"The file '{file_path}' does not exist.")

The file '/content/drive/My Drive/60_model_bs_256_frame2.h5' does not exist.


In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import os

# Constants
IMAGE_HEIGHT, IMAGE_WIDTH, SEQUENCE_LENGTH = 256, 256, 60
CLASSES_LIST = ["Normal", "Shoplifting"]

def preprocess_frame(frame):
    resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
    return resized_frame / 255.0

def predict_per_frame(video_path, model, output_video_path, threshold=0.70):
    video_capture = cv2.VideoCapture(video_path)
    frame_count = 0

    # Get video properties
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Output codec for mp4
    frame_rate = video_capture.get(cv2.CAP_PROP_FPS)  # Frame rate of the input video
    width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Set up the VideoWriter to save the output video
    out = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (width, height))

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        # Preprocess frame for prediction
        processed_frame = preprocess_frame(frame)
        input_data = np.expand_dims(processed_frame, axis=0)  # Add batch dimension

        # Predict on the frame
        prediction = model.predict(input_data, verbose=0)  # Suppress output
        max_probability = np.max(prediction)
        predicted_class = np.argmax(prediction)
        predicted_label = (
            CLASSES_LIST[predicted_class] if max_probability >= threshold else "Normal"
        )

        print(f"Frame {frame_count}: Predicted: {predicted_label} (Probability: {max_probability:.2f})")

        # Annotate the frame with prediction
        text = f"Prediction: {predicted_label}"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1.5
        font_thickness = 2
        text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
        text_width, text_height = text_size
        text_x, text_y = 10, 40  # Top-left corner of the text

        # Draw a black rectangle as the background for the text
        cv2.rectangle(frame, (text_x - 5, text_y - text_height - 5),
                      (text_x + text_width + 5, text_y + 5), (0, 0, 0), -1)

        # Put the white text on top of the rectangle
        cv2.putText(frame, text, (text_x, text_y), font, font_scale, (255, 255, 255), font_thickness)

        # Write the annotated frame to the output video
        out.write(frame)

        frame_count += 1

        # Exit on 'q' (not necessary for saving, but useful for manual control)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    out.release()  # Release the output video writer
    cv2.destroyAllWindows()


def main():
    # Path to the input video in Google Drive
    video_path = '/content/drive/MyDrive/FYP Dataset/Shoplifting/Shoplifting-92.mp4'

    # Create an output folder named "output_videos" in Google Drive
    output_folder = '/content/drive/MyDrive/output_videos_movienet'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Created output folder: {output_folder}")

    # Define the output video path
    output_video_path = os.path.join(output_folder, 'Shoplifting-92_output2.mp4')

    # Load the trained model
    model = tf.keras.models.load_model('/content/drive/My Drive/60_model_bs_256_frame2.h5')  # Update with your model path

    # Process the video and save results
    predict_per_frame(video_path, model, output_video_path, threshold=0.70)
    print(f"Output video saved at: {output_video_path}")


if __name__ == "__main__":
    main()


ValueError: No model config found in the file at /content/drive/My Drive/60_model_bs_256_frame2.h5.